## Changelog
* V1.0 
    - Adding masking layer
* V1.1
    - Scaling dataset using RobustScaler

In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import RobustScaler
from pandas import DataFrame, Series
from keras.layers import Dense, LSTM, SimpleRNN, Masking
from keras.models import Sequential
from keras.preprocessing import sequence
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

/home/iss/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
data = DataFrame.from_csv("./testosterone_levels.csv")

In [3]:
tail_length = 1

In [4]:
# adding seq_ix to data
seq_ix = [list(np.arange(i)) for i in data.groupby("Person").size().values]
seq_ix = [item for sub in seq_ix for item in sub]
data.index = pd.MultiIndex.from_arrays([data.index.values, seq_ix], names=['Person', 'seq_ix'])

In [5]:
data.fillna(method='pad', inplace=True)

In [6]:
# scaling data
scaler = RobustScaler()
scaled_data = scaler.fit_transform(data)
scaled_data = DataFrame(scaled_data, index = data.index)

In [7]:
# dividing dataset for creating labels (train-test validation)
person_heads = []
person_tails = []
for id, details in scaled_data.groupby("Person"):
    head = details.head(len(details) - tail_length).values
    tail = details.tail(tail_length).values
    person_heads.append(head)
    person_tails.append(tail)
person_tails = np.array(person_tails)

In [8]:
# padding data
person_heads = sequence.pad_sequences(np.array(person_heads), dtype='float32')

In [9]:
# spliting train/test data
x_train, x_test, y_train, y_test = train_test_split(person_heads, person_tails, test_size=.2)

In [10]:
# creating model
model = Sequential()
model.add(Masking(input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1))
from keras.optimizers import adam
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 17, 5)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 17, 100)           42400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 122,901
Trainable params: 122,901
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
#defining callbacks
from keras.callbacks import EarlyStopping, TensorBoard
tensorboard_callback = TensorBoard(log_dir="./logs/2_LSTM_100_run26_maskingLayer_scaled_batch2" , histogram_freq=1, batch_size=1)
earlystopping_callback = EarlyStopping(min_delta=.001 , patience=20, mode='min', verbose=1)

In [12]:
history = model.fit(x_train, y_train[:,:,-1], batch_size=2, epochs=100, validation_data=(x_test,y_test[:,:,-1]), callbacks=[tensorboard_callback])

Train on 110 samples, validate on 28 samples
Epoch 1/100
110/110 [==============================] - 2s - loss: 2.2089 - mean_absolute_error: 0.7443 - val_loss: 0.5701 - val_mean_absolute_error: 0.4539
Epoch 2/100
110/110 [==============================] - 2s - loss: 0.8647 - mean_absolute_error: 0.4431 - val_loss: 0.3501 - val_mean_absolute_error: 0.3111
Epoch 3/100
110/110 [==============================] - 2s - loss: 1.0468 - mean_absolute_error: 0.6047 - val_loss: 0.7071 - val_mean_absolute_error: 0.5048
Epoch 4/100
110/110 [==============================] - 2s - loss: 0.7384 - mean_absolute_error: 0.4331 - val_loss: 0.3355 - val_mean_absolute_error: 0.3618
Epoch 5/100
110/110 [==============================] - 2s - loss: 0.4948 - mean_absolute_error: 0.4001 - val_loss: 0.2914 - val_mean_absolute_error: 0.3308
Epoch 6/100
110/110 [==============================] - 2s - loss: 0.2620 - mean_absolute_error: 0.2874 - val_loss: 0.2501 - val_mean_absolute_error: 0.3033
Epoch 7/100
110/110